In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES']=''

import sys;
sys.path.append('..')

In [2]:
import json
from pathlib import Path

In [3]:
# preprocess ods_data
from tqdm import tqdm_notebook as tqdm

path = Path('/home/kuratov/data/ods_shards/ods_shards/')
files = sorted(list(path.glob('*json')))

total_articles = 0
filtered_articles = 0

articles = []

for f in tqdm(files):
    dialogs = json.load(f.open('r'))
    for d in dialogs:
        total_articles += 1
        dialog = ' '.join(d) 
        dialog = dialog.replace('<speaker1>', '<speaker1> ').replace('<speaker2>', '<speaker2> ')
        if len(dialog) == 0:
            filtered_articles += 1
            continue
        articles += [{'text': dialog}]
print(f'{filtered_articles}/{total_articles}')

/home/kuratov/anaconda3/envs/tf2.0/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':



0/59545927


In [4]:
len(articles)

59545927

In [5]:
articles[0]

{'text': '<speaker1> За вами очень хорошо наблюдают, а дома вы на прослушке. <speaker2> Вам и это известно?'}

In [6]:
59545927 / 320

186081.021875

In [7]:
# save shards, with valid shard 0.5 of train_shard size

shards_path = '/home/kuratov/data/ods_shards/preprocessed_shards_train'

if not Path(shards_path).exists():
    Path(shards_path).mkdir(parents=True)

import random, math
#random.shuffle(articles)
n_train_shards = 320  # (we possibly have to take n_gpus shards for validation, so we need more shards)
n_valid_shards = 1 #  add +1 for validation, only 1 works currently
valid_shard_size = 0.5 # half of the train shard

shard_size = math.ceil(len(articles) / (n_train_shards + n_valid_shards))
shard_size = math.ceil((len(articles) - shard_size * valid_shard_size) / n_train_shards)

for i, start in enumerate(tqdm(range(0, len(articles), shard_size))):
    with (Path(shards_path) / f'shard_{i:03d}.jsonl').open('w', encoding='utf8') as fout:
        #print(start, start + shard_size)
        for j in range(start, min(len(articles), start+shard_size)):
            jsoned = json.dumps(articles[j])
            try:
                tmp = json.loads(jsoned)
            except json.JSONDecodeError as e:
                print(f'skipping article {j} in shard {i} cause of JSONDecodeError')
                continue
            fout.write(jsoned+'\n')

/home/kuratov/anaconda3/envs/tf2.0/lib/python3.7/site-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
